# Assignment — Knowledge Graph Embeddings

In this assignment we will see how to use the TorchKDE library for building knowledge graphs and its embeddings.

In [ ]:
import numpy as np
import pandas as pd
from hashlib import sha1
import requests

### Task 1 . Dataset exploration (4 points)

To begin with we are going to need a knowledge graph, so let us load a standard knowledge graph dataset called _Freebase-15k-237_.

In [ ]:
url = 'https://raw.githubusercontent.com/vpozdnyakov/network_science_assignments/master/assignment_knowledge_graph_embeddings/freebase-237-merged-and-remapped.csv'
open('freebase-237-merged-and-remapped.csv', 'wb').write(requests.get(url).content)


In [ ]:
df = pd.read_csv('freebase-237-merged-and-remapped.csv', 
                 names=['h', 'r', 't'])
df = df[~df.h.str.startswith('/') & ~df.t.str.startswith('/')]
df[::1001].head()

There is h — head (also subject), r — relation (also predicat), t — tail (also object). The shape of the dataset is

In [ ]:
df.shape

Let us check the number of unique entities and unique relations.

Write a funtion `n_ent_rel` that takes a dataset and returns a number of unique entities and unique relations.


In [ ]:
def n_ent_rel(df):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
n_ent, n_rel = n_ent_rel(df)
assert sha1((str(n_ent + n_ent)).encode('utf-8')).hexdigest() == '2a5452eba9870d4f95a77176bbab9b5a862bda60'
n_ent, n_rel

Let us try to find some facts in this dataset. For example, what is Harrison Ford's nationality? ("harrison ford" in the dataset)

Write a function `harrison_ford_nationality` that takes a dataset and returns the nationality.

_Hint: use pandas.Series.str.contains method_

In [ ]:
def harrison_ford_nationality(df):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert sha1(harrison_ford_nationality(df).encode('utf-8')).hexdigest() == '2da4d0b37c841952f402a493ade05c98f19f4a14'
harrison_ford_nationality(df)

More tricky question: who are film directors of movies where Harrison Ford was?

Write a function `made_films_with_harrison_ford` that returns a set of directors' names.

In [ ]:
def made_films_with_harrison_ford(df):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
directors = made_films_with_harrison_ford(df)
assert sha1(str(sorted(directors)).encode('utf-8')).hexdigest() == 'c317bd28b873d3ae1a0444d9351d9d0ff5f2f348'
directors

### Task 2. ComplEx embedding model (3 points)

We will use TorchKDE — a Python module for knowledge graph (KG) embedding relying solely on Pytorch. Let us install it.

In [ ]:
!pip install torchkge

In [ ]:
import torch
import torchkge
from torchkge.models import ComplExModel, TransEModel
from torchkge.utils import Trainer, MarginLoss
from torchkge.evaluation import LinkPredictionEvaluator, TripletClassificationEvaluator

ComplEx scoring function is based on the trilinear Hermitian dot product
$$f=\text{Re}\left(\langle w_\text{r}, e_\text{h}, \overline{e}_\text{t}  \rangle\right)$$
where 
* $w_\text{r}$, $e_\text{h}$, ${e}_\text{t}$ are embeddings for relation, head, tail
* $\text{Re}(x)$ is a real part of $x$
* $\overline x = \text{Re}(x) - i\text{Im}(x)$

Let us look at the WikiDataSet that presents country-specific subgraphs of Wikidata.

In [ ]:
url = 'https://raw.githubusercontent.com/vpozdnyakov/network_science_assignments/master/assignment_knowledge_graph_embeddings/countries_edges.tsv'
open('countries_edges.tsv', 'wb').write(requests.get(url).content)
url = 'https://raw.githubusercontent.com/vpozdnyakov/network_science_assignments/master/assignment_knowledge_graph_embeddings/countries_entities.tsv'
open('countries_entities.tsv', 'wb').write(requests.get(url).content)
url = 'https://raw.githubusercontent.com/vpozdnyakov/network_science_assignments/master/assignment_knowledge_graph_embeddings/countries_relations.tsv'
open('countries_relations.tsv', 'wb').write(requests.get(url).content)


In [ ]:
edges = pd.read_csv('countries_edges.tsv', sep='	').values
entity_labels = pd.read_csv('countries_entities.tsv', sep='	', index_col=0).label.values
relation_labels = pd.read_csv('countries_relations.tsv', sep='	', index_col=0).label.values

In [ ]:
edges_labeled = np.stack([entity_labels[edges[:, 0]], 
                          entity_labels[edges[:, 1]], 
                          relation_labels[edges[:, 2]]], axis=1)

In [ ]:
pd.DataFrame(edges_labeled[10::1002], columns=['h', 't', 'r'])[['h', 'r', 't']]

Our model minimizes Margin loss

$\mathcal L = \max\{0, \gamma - f(h,r,t) + f(h',r',t')\}$ where
* $\gamma$ is the margin (defined at initialization),
* $f(h,r,t)$ is the score of a true fact and
* $f(h',r',t')$ is the score of the associated negative fact.

Let us convert our dataset into a graph

In [ ]:
kg = torchkge.KnowledgeGraph(
    pd.DataFrame(edges_labeled, columns=['from', 'to', 'rel']))

Next, let us split the dataset into train and test set. What differs from the standard method of randomly sampling N points to make up our test set, is that our data points are two entities linked by some relationship, and we need to take care to ensure that all entities are represented in train and test sets by at least one triple.



In [ ]:
kg_train, kg_test = kg.split_kg()

Next, go through the parameters to understand what’s going on:
* `emb_dim` is the dimensionality of the embedding space
* `margin` is the $\gamma$ parameter of Margin loss

In [ ]:
emb_dim = 100
margin = 0.5
n_epochs = 150
batch_size = 3000

In [ ]:
model = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel)

In [ ]:
criterion = MarginLoss(margin)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

In [ ]:
trainer = Trainer(model, criterion, kg_train, n_epochs, batch_size,
                  optimizer=optimizer, sampling_type='bern')
trainer.run()

Let us evaluate our model

In [ ]:
evaluator = LinkPredictionEvaluator(model, kg_test)
evaluator.evaluate(b_size=32, k_max=10)
evaluator.print_results()

* `Hit@k` indicates how many times in average a true triple was ranked in the top-k. Therefore, on average, we guessed the correct subject or object 25% of the time when considering the top-10 better ranked triples. The choice of which k makes more sense depends on the application.

* `Mean Rank` is a mean rank of the true entity when replacing alternatively head and tail in any fact of the dataset.

* `MRR` is an average of mean recovery rank for head and tail replacement.

* Filtered metrics are given with replacing alternatively head and tail in any fact of the dataset.

Let us find a nearest neighbors of Belgium using embedding space.

Write a function `similar_countries` that takes a name of country, graph and model and returns a list with names of nearest countries. Use `model.get_embeddings()`.

In [ ]:
def similar_countries(name, kg, model):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
similar = similar_countries('Belgium', kg, model)
assert 'Netherlands' in similar
similar

### Task 3. Predicting New Links (3 points)

Let us ckeck these facts:
1. Belgium shares border with France
2. Belgium shares border with Switzerland
3. Belgium shares border with Nigeria

Write a function `score_facts` that takes a model, a graph and returns 3 values of scoring function for each fact. Use `model.scoring_function`.

In [ ]:
def score_facts(model, kg):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
scores = score_facts(model, kg)
assert scores[0] > scores[1] > scores[2]
scores